- In this Notebook, I am trying to integrate the training data from the backscore and rest of the period from prj-prod-dataplatform.audit_balance.ml_model_run_details table. 
- In this I will compare the training period with each month of test period.

# Define Libraries

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.
# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import os
import tempfile
import time
from datetime import datetime
import uuid
import joblib
import uuid

import gcsfs
import duckdb as dd
import pickle
import joblib
from typing import Union
import io

path = r'C:\Users\Dwaipayan\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')
os.environ["GOOGLE_CLOUD_PROJECT"] = "prj-prod-dataplatform"
# %% [markdown]
## Configure Settings
# Set options or configurations as needed
pd.set_option('display.max_columns', None)
pd.set_option("Display.max_rows", 100)

# Function

## expand_calc_features

In [2]:
import pandas as pd
import json

def expand_calc_features(df):
    """
    Expand the calcFeatures JSON column into separate columns and return the complete DataFrame.
    
    Parameters:
    df (pd.DataFrame): Input DataFrame with calcFeatures column containing JSON data
    
    Returns:
    pd.DataFrame: Expanded DataFrame with all original columns plus JSON features as separate columns
    """
    
    # Make a copy to avoid modifying the original DataFrame
    df_expanded = df.copy()
    
    # Parse the calcFeatures JSON column
    calc_features_list = []
    
    for idx, calc_features_str in enumerate(df['calcFeatures']):
        try:
            # Parse the JSON string
            features_dict = json.loads(calc_features_str.replace("'", '"'))  # Replace single quotes with double quotes for valid JSON
            calc_features_list.append(features_dict)
        except (json.JSONDecodeError, AttributeError) as e:
            # If parsing fails, create an empty dict and print warning
            print(f"Warning: Could not parse calcFeatures at index {idx}: {e}")
            calc_features_list.append({})
    
    # Create DataFrame from the parsed JSON data
    calc_features_df = pd.DataFrame(calc_features_list)
    
    # Add prefix to JSON-derived columns to avoid conflicts
    calc_features_df = calc_features_df.add_prefix('calc_')
    
    # Reset index to ensure proper alignment
    df_expanded = df_expanded.reset_index(drop=True)
    calc_features_df = calc_features_df.reset_index(drop=True)
    
    # Combine original DataFrame with expanded calcFeatures
    result_df = pd.concat([df_expanded, calc_features_df], axis=1)
    
    return result_df


## expand_calc_features_robust

In [3]:
import pandas as pd
import json

def expand_calc_features_robust(df):
    """
    Expand the calcFeatures JSON column into separate columns with better error handling.
    
    Parameters:
    df (pd.DataFrame): Input DataFrame with calcFeatures column containing JSON data
    
    Returns:
    pd.DataFrame: Expanded DataFrame with all original columns plus JSON features as separate columns
    """
    
    # Make a copy to avoid modifying the original DataFrame
    df_expanded = df.copy()
    
    # Parse the calcFeatures JSON column
    calc_features_data = []
    
    for idx, row in df.iterrows():
        calc_features_str = row['calcFeatures']
        
        if pd.isna(calc_features_str) or calc_features_str == '':
            calc_features_data.append({})
            continue
            
        try:
            # Clean the string and parse JSON
            cleaned_str = calc_features_str.replace("'", '"').replace('None', 'null').replace('True', 'true').replace('False', 'false')
            features_dict = json.loads(cleaned_str)
            calc_features_data.append(features_dict)
        except Exception as e:
            print(f"Warning: Could not parse calcFeatures at index {idx}: {e}")
            print(f"Problematic string: {calc_features_str[:100]}...")  # Print first 100 chars
            calc_features_data.append({})
    
    # Create DataFrame from the parsed JSON data
    calc_features_df = pd.DataFrame(calc_features_data)
    
    # Add prefix to JSON-derived columns to avoid conflicts with existing columns
    calc_features_df = calc_features_df.add_prefix('feat_')
    
    # Combine DataFrames
    result_df = pd.concat([df_expanded, calc_features_df], axis=1)
    
    print(f"Original DataFrame shape: {df.shape}")
    print(f"Expanded DataFrame shape: {result_df.shape}")
    print(f"Added {len(calc_features_df.columns)} new columns from calcFeatures")
    
    return result_df

## PSI Functions new

In [4]:
## Updated on 27-10-2025 - Modified for Training Period Baseline
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

def identify_feature_types(df: pd.DataFrame, feature_list: List[str]) -> Dict[str, List[str]]:
    """
    Identify categorical and numerical features from the feature list.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to classify
    
    Returns:
    --------
    Dict with 'categorical' and 'numerical' keys containing respective feature lists
    """
    categorical_features = []
    numerical_features = []
    
    for feature in feature_list:
        if feature not in df.columns:
            print(f"Warning: Feature '{feature}' not found in dataframe")
            continue
            
        # Check if feature is numeric
        if pd.api.types.is_numeric_dtype(df[feature]):
            # If unique values are less than 15 and all integers, treat as categorical
            unique_vals = df[feature].nunique()
            if unique_vals < 15 and df[feature].dropna().apply(lambda x: x == int(x) if isinstance(x, (int, float)) else False).all():
                categorical_features.append(feature)
            else:
                numerical_features.append(feature)
        else:
            categorical_features.append(feature)
    
    return {
        'categorical': categorical_features,
        'numerical': numerical_features
    }


def create_bins_for_features(df: pd.DataFrame, 
                             numerical_features: List[str],
                             categorical_features: List[str],
                             train_period_df: pd.DataFrame) -> Dict:
    """
    Create bins for numerical features (deciles) and categorical features (top 6 + others)
    based on the entire training period data.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Full input dataframe
    numerical_features : List[str]
        List of numerical features
    categorical_features : List[str]
        List of categorical features
    train_period_df : pd.DataFrame
        Training period dataframe (June 2024 to March 2025)
    
    Returns:
    --------
    Dictionary containing binning information for each feature
    """
    binning_info = {}
    
    # Create bins for numerical features (deciles) using training period
    for feature in numerical_features:
        valid_data = train_period_df[feature].dropna()
        
        if len(valid_data) == 0:
            binning_info[feature] = {'type': 'numerical', 'bins': None, 'bin_ranges': {}}
            continue
        
        # Create decile bins
        try:
            bins = np.percentile(valid_data, np.arange(0, 101, 10))
            # Remove duplicates and sort
            bins = np.unique(bins)
            
            # Add infinity edges to capture all values
            bins = bins.copy()
            bins[0] = -np.inf
            bins[-1] = np.inf
            
            # Warn if fewer bins than expected due to low variance
            if len(bins) < 11:  # 11 edges = 10 bins
                print(f"Warning: Feature '{feature}' has only {len(bins)-1} bins due to low variance in training period")
            
            # Create bin ranges dictionary
            bin_ranges = {}
            for i in range(len(bins)-1):
                bin_name = f"Bin_{i+1}"
                bin_ranges[bin_name] = {
                    'min': bins[i],
                    'max': bins[i+1],
                    'range_str': f"[{bins[i]:.2f}, {bins[i+1]:.2f}]" if not np.isinf(bins[i]) and not np.isinf(bins[i+1]) else f"({bins[i]}, {bins[i+1]})"
                }
            print(f"Bin Ranges for feature '{feature}': {bin_ranges}")
            binning_info[feature] = {
                'type': 'numerical',
                'bins': bins,
                'bin_ranges': bin_ranges
            }
        except Exception as e:
            print(f"Error creating bins for feature '{feature}': {str(e)}")
            binning_info[feature] = {'type': 'numerical', 'bins': None, 'bin_ranges': {}}
    
    # Create bins for categorical features (top 6 + others) using training period
    for feature in categorical_features:
        value_counts = train_period_df[feature].value_counts()
        unique_categories = value_counts.index.tolist()
        print(f"Unique categories:{unique_categories}")
        
        if len(unique_categories) <= 6:
            # Treat each category as a separate bin
            top_categories = unique_categories
        else:
            # Use top 6 categories only
            top_categories = value_counts.nlargest(6).index.tolist()

        
        print(f"Top categories for feature '{feature}': {top_categories}")
        
        binning_info[feature] = {
                'type': 'categorical',
                'top_categories': top_categories,
                'bin_ranges': {}  # No ranges for categorical
            }

    
    return binning_info


def apply_binning(df: pd.DataFrame, 
                  feature: str, 
                  binning_info: Dict) -> pd.Series:
    """
    Apply binning to a feature based on binning information.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature : str
        Feature name
    binning_info : Dict
        Binning information for the feature
    
    Returns:
    --------
    pd.Series with binned values
    """
    if binning_info['type'] == 'numerical':
        if binning_info['bins'] is None:
            return pd.Series(['Missing'] * len(df), index=df.index)
        
        bins = binning_info['bins']
        labels = [f"Bin_{i+1}" for i in range(len(bins)-1)]
        
        binned = pd.cut(df[feature], 
                       bins=bins, 
                       labels=labels, 
                       include_lowest=True,
                       duplicates='drop')
        
        # Handle nulls - convert to string and then replace
        binned = binned.astype(str)
        binned[df[feature].isna()] = 'Missing'
        
        return binned
    
    else:  # categorical
        top_cats = binning_info['top_categories']
        
        # Convert categorical to object if needed to avoid category errors
        if pd.api.types.is_categorical_dtype(df[feature]):
            feature_data = df[feature].astype(str)
        else:
            feature_data = df[feature].astype(str)
        
        # Replace NaN string representation with 'Missing'
        feature_data = feature_data.replace('nan', 'Missing')
        
        # Apply binning logic
        binned = feature_data.apply(lambda x: x if x in top_cats else ('Missing' if x == 'Missing' else 'Others'))
        
        return binned


def calculate_psi(expected_pct: pd.Series, 
                  actual_pct: pd.Series, 
                  epsilon: float = 0.0001) -> float:
    """
    Calculate Population Stability Index with proper epsilon handling and renormalization.
    
    Parameters:
    -----------
    expected_pct : pd.Series
        Expected (baseline) percentages
    actual_pct : pd.Series
        Actual percentages
    epsilon : float
        Small value to avoid log(0)
    
    Returns:
    --------
    PSI value
    """
    # Align indices
    all_bins = expected_pct.index.union(actual_pct.index)
    expected_pct = expected_pct.reindex(all_bins, fill_value=0)
    actual_pct = actual_pct.reindex(all_bins, fill_value=0)
    
    # Only add epsilon where values are zero
    expected_pct = expected_pct.apply(lambda x: epsilon if x == 0 else x)
    actual_pct = actual_pct.apply(lambda x: epsilon if x == 0 else x)
    
    # Renormalize to ensure they sum to 1 after adding epsilon
    expected_pct = expected_pct / expected_pct.sum()
    actual_pct = actual_pct / actual_pct.sum()
    
    # Calculate PSI
    psi_value = np.sum((actual_pct - expected_pct) * np.log(actual_pct / expected_pct))
    
    return psi_value


def calculate_month_on_month_psi(df: pd.DataFrame,
                                 feature_list: List[str],
                                 segment_columns: List[str],
                                 month_col: str = 'Application_month',
                                 data_selection_col: str = 'Data_selection',
                                 account_id_col: str = 'digitalLoanAccountId') -> pd.DataFrame:
    """
    Calculate PSI for each feature comparing training period (June 2024 to March 2025) 
    vs each month after March 2025, overall and by segments.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to calculate PSI for
    segment_columns : List[str]
        List of segment columns
    month_col : str
        Name of month column
    data_selection_col : str
        Name of data selection column (identifies train period)
    account_id_col : str
        Name of account ID column for counting distinct accounts
    
    Returns:
    --------
    pd.DataFrame with PSI values with one row per feature-month-segment combination
    """
    # Create a copy to avoid modifying original
    df = df.copy()
    
    # Identify training and test periods
    train_df = df[df[data_selection_col] == 'Train'].copy()
    test_df = df[df[data_selection_col] != 'Train'].copy()
    
    if len(train_df) == 0:
        raise ValueError("No training data found. Check Data_selection column.")
    
    print(f"Training period: {train_df[month_col].min()} to {train_df[month_col].max()}")
    print(f"Test period: {test_df[month_col].min()} to {test_df[month_col].max()}")
    
    # Identify feature types
    feature_types = identify_feature_types(df, feature_list)
    
    # Create binning strategy based on training period
    binning_info = create_bins_for_features(
        df, 
        feature_types['numerical'],
        feature_types['categorical'],
        train_df
    )
    
    # Get sorted test months
    test_months = sorted(test_df[month_col].unique())
    
    results = []
    
    # Calculate overall PSI
    for feature in feature_list:
        if feature not in df.columns:
            continue
        
        # Apply binning to entire dataset
        df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        # print(f"Feature binned {df[f'{feature}_binned']}")
        # Get training period distribution (baseline)
        train_baseline = df[df[data_selection_col] == 'Train'][f'{feature}_binned'].value_counts(normalize=True)
        
        # Calculate PSI for each test month
        for month in test_months:
            actual_dist = df[df[month_col] == month][f'{feature}_binned'].value_counts(normalize=True)
            psi_value = calculate_psi(train_baseline, actual_dist)

            # Calculate average percentages across all bins
            expected_avg_pct = train_baseline.mean() * 100
            actual_avg_pct = actual_dist.mean() * 100
            
            # # Count distinct accounts for segment
            # base_segment_count = train_segment[account_id_col].nunique()
            # actual_segment_count = actual_segment[account_id_col].nunique()
                    
            
            results.append({
                'Feature': feature,
                'Feature_Type': binning_info[feature]['type'],
                'Segment_Column': 'Overall',
                'Segment_Value': 'All',
                'Month': f"{month}",
                'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                'Current_Month': month,
                'Expected_Percentage': expected_avg_pct,
                'Actual_Percentage': actual_avg_pct,
                'PSI': psi_value
            })
    
    # Calculate PSI by segments
    for segment_col in segment_columns:
        if segment_col not in df.columns:
            continue
        
        segments = df[segment_col].dropna().unique()
        
        for segment_val in segments:
            segment_df = df[df[segment_col] == segment_val]
            
            for feature in feature_list:
                if feature not in df.columns:
                    continue
                
                # Get training period distribution for segment
                train_segment = segment_df[segment_df[data_selection_col] == 'Train']
                if len(train_segment) == 0:
                    continue
                
                train_baseline = train_segment[f'{feature}_binned'].value_counts(normalize=True)
                
                # Calculate PSI for each test month
                for month in test_months:
                    actual_segment = segment_df[segment_df[month_col] == month]
                    if len(actual_segment) == 0:
                        continue
                    
                    actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)
                    psi_value = calculate_psi(train_baseline, actual_dist)
                    
                    # Calculate average percentages across all bins
                    expected_avg_pct = train_baseline.mean() * 100
                    actual_avg_pct = actual_dist.mean() * 100
                    
                    # Count distinct accounts for segment
                    base_segment_count = train_segment[account_id_col].nunique()
                    actual_segment_count = actual_segment[account_id_col].nunique()
                    
                    results.append({
                        'Feature': feature,
                        'Feature_Type': binning_info[feature]['type'],
                        'Segment_Column': segment_col,
                        'Segment_Value': segment_val,
                        'Month': f"{month}",
                        'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                        'Current_Month': month,
                        'Base_Count': base_segment_count,
                        'Actual_Count': actual_segment_count,
                        'Expected_Percentage': expected_avg_pct,
                        'Actual_Percentage': actual_avg_pct,
                        'PSI': psi_value
                    })
    
    return pd.DataFrame(results)


def calculate_bin_level_psi(df: pd.DataFrame,
                            feature_list: List[str],
                            segment_columns: List[str],
                            month_col: str = 'Application_month',
                            data_selection_col: str = 'Data_selection',
                            account_id_col: str = 'digitalLoanAccountId') -> pd.DataFrame:
    """
    Calculate bin-level PSI for each feature comparing training period 
    vs each month after March 2025, overall and by segments.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to calculate PSI for
    segment_columns : List[str]
        List of segment columns
    month_col : str
        Name of month column
    data_selection_col : str
        Name of data selection column
    account_id_col : str
        Name of account ID column for counting distinct accounts
    
    Returns:
    --------
    pd.DataFrame with bin-level PSI details including bin ranges
    """
    # Create a copy to avoid modifying original
    df = df.copy()
    
    # Identify training and test periods
    train_df = df[df[data_selection_col] == 'Train'].copy()
    test_df = df[df[data_selection_col] != 'Train'].copy()
    
    if len(train_df) == 0:
        raise ValueError("No training data found. Check Data_selection column.")
    
    print(f"Training period: {train_df[month_col].min()} to {train_df[month_col].max()}")
    print(f"Test period: {test_df[month_col].min()} to {test_df[month_col].max()}")
    
    # Identify feature types
    feature_types = identify_feature_types(df, feature_list)
    
    # Create binning strategy based on training period
    binning_info = create_bins_for_features(
        df, 
        feature_types['numerical'],
        feature_types['categorical'],
        train_df
    )
    
    # Get sorted test months
    test_months = sorted(test_df[month_col].unique())
    
    results = []
    epsilon = 0.0001
    
    # Calculate overall bin-level PSI
    for feature in feature_list:
        if feature not in df.columns:
            continue
        
        # Apply binning to entire dataset
        df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        print(df[f'{feature}_binned'])
        
        # Get training period distribution (baseline)
        train_baseline = df[df[data_selection_col] == 'Train'][f'{feature}_binned'].value_counts(normalize=True)
        
        # Calculate bin-level PSI for each test month
        for month in test_months:
            month_data = df[df[month_col] == month]
            actual_dist = month_data[f'{feature}_binned'].value_counts(normalize=True)
            
            # Count distinct accounts
            base_count = df[df[data_selection_col] == 'Train'][account_id_col].nunique()
            actual_count = month_data[account_id_col].nunique()
            
            # Get all bins
            all_bins = train_baseline.index.union(actual_dist.index)
            
            for bin_name in all_bins:
                # Simplified epsilon logic - no redundancy
                expected_pct = train_baseline.get(bin_name, 0)
                actual_pct = actual_dist.get(bin_name, 0)
                
                # Add epsilon only if zero
                expected_pct = epsilon if expected_pct == 0 else expected_pct
                actual_pct = epsilon if actual_pct == 0 else actual_pct
                
                # Calculate bin-level PSI
                bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct)
                
                # Get bin range information
                bin_ranges = binning_info[feature]['bin_ranges']
                if bin_name in bin_ranges:
                    bin_min = bin_ranges[bin_name]['min']
                    bin_max = bin_ranges[bin_name]['max']
                    bin_range = bin_ranges[bin_name]['range_str']
                else:
                    # For categorical or special bins (Missing, Others)
                    bin_min = None
                    bin_max = None
                    bin_range = bin_name
                
                results.append({
                    'Feature': feature,
                    'Feature_Type': binning_info[feature]['type'],
                    'Segment_Column': 'Overall',
                    'Segment_Value': 'All',
                    'Month': f"{month}",
                    'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                    'Current_Month': month,
                    'Base_Count': base_count,
                    'Actual_Count': actual_count,
                    'Bin': bin_name,
                    'Bin_Range': bin_range,
                    'Bin_Min': bin_min,
                    'Bin_Max': bin_max,
                    'Base_Percentage': (train_baseline.get(bin_name, 0) * 100),
                    'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
                    'Bin_PSI': bin_psi
                })
    
    # Calculate bin-level PSI by segments
    for segment_col in segment_columns:
        if segment_col not in df.columns:
            continue
        
        segments = df[segment_col].dropna().unique()
        
        for segment_val in segments:
            segment_df = df[df[segment_col] == segment_val]
            
            for feature in feature_list:
                if feature not in df.columns:
                    continue
                
                # Get training period distribution for segment
                train_segment = segment_df[segment_df[data_selection_col] == 'Train']
                if len(train_segment) == 0:
                    continue
                
                train_baseline = train_segment[f'{feature}_binned'].value_counts(normalize=True)
                
                # Calculate bin-level PSI for each test month
                for month in test_months:
                    actual_segment = segment_df[segment_df[month_col] == month]
                    if len(actual_segment) == 0:
                        continue
                    
                    actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)
                    
                    # Count distinct accounts for segment
                    base_segment_count = train_segment[account_id_col].nunique()
                    actual_segment_count = actual_segment[account_id_col].nunique()
                    
                    # Get all bins
                    all_bins = train_baseline.index.union(actual_dist.index)
                    
                    for bin_name in all_bins:
                        # Simplified epsilon logic - no redundancy
                        expected_pct = train_baseline.get(bin_name, 0)
                        actual_pct = actual_dist.get(bin_name, 0)
                        
                        # Add epsilon only if zero
                        expected_pct = epsilon if expected_pct == 0 else expected_pct
                        actual_pct = epsilon if actual_pct == 0 else actual_pct
                        
                        # Calculate bin-level PSI
                        bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct)
                        
                        # Get bin range information
                        bin_ranges = binning_info[feature]['bin_ranges']
                        if bin_name in bin_ranges:
                            bin_min = bin_ranges[bin_name]['min']
                            bin_max = bin_ranges[bin_name]['max']
                            bin_range = bin_ranges[bin_name]['range_str']
                        else:
                            # For categorical or special bins (Missing, Others)
                            bin_min = None
                            bin_max = None
                            bin_range = bin_name
                        
                        results.append({
                            'Feature': feature,
                            'Feature_Type': binning_info[feature]['type'],
                            'Segment_Column': segment_col,
                            'Segment_Value': segment_val,
                            'Month': f"{month}",
                            'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                            'Current_Month': month,
                            'Base_Count': base_segment_count,
                            'Actual_Count': actual_segment_count,
                            'Bin': bin_name,
                            'Bin_Range': bin_range,
                            'Bin_Min': bin_min,
                            'Bin_Max': bin_max,
                            'Base_Percentage': (train_baseline.get(bin_name, 0) * 100),
                            'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
                            'Bin_PSI': bin_psi
                        })
    
    return pd.DataFrame(results)

In [5]:
# ## updated on 21-10-2025
# import pandas as pd
# import numpy as np
# from typing import List, Dict, Tuple
# import warnings
# warnings.filterwarnings('ignore')

# def identify_feature_types(df: pd.DataFrame, feature_list: List[str]) -> Dict[str, List[str]]:
#     """
#     Identify categorical and numerical features from the feature list.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     feature_list : List[str]
#         List of features to classify
    
#     Returns:
#     --------
#     Dict with 'categorical' and 'numerical' keys containing respective feature lists
#     """
#     categorical_features = []
#     numerical_features = []
    
#     for feature in feature_list:
#         if feature not in df.columns:
#             print(f"Warning: Feature '{feature}' not found in dataframe")
#             continue
            
#         # Check if feature is numeric
#         if pd.api.types.is_numeric_dtype(df[feature]):
#             # If unique values are less than 15 and all integers, treat as categorical
#             unique_vals = df[feature].nunique()
#             if unique_vals < 15 and df[feature].dropna().apply(lambda x: x == int(x) if isinstance(x, (int, float)) else False).all():
#                 categorical_features.append(feature)
#             else:
#                 numerical_features.append(feature)
#         else:
#             categorical_features.append(feature)
    
#     return {
#         'categorical': categorical_features,
#         'numerical': numerical_features
#     }


# def create_bins_for_features(df: pd.DataFrame, 
#                              numerical_features: List[str],
#                              categorical_features: List[str],
#                              base_month: str,
#                              month_col: str = 'Application_month') -> Dict:
#     """
#     Create bins for numerical features (deciles) and categorical features (top 6 + others).
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     numerical_features : List[str]
#         List of numerical features
#     categorical_features : List[str]
#         List of categorical features
#     base_month : str
#         Base month to determine binning strategy
#     month_col : str
#         Name of month column
    
#     Returns:
#     --------
#     Dictionary containing binning information for each feature
#     """
#     base_df = df[df[month_col] == base_month].copy()
#     binning_info = {}
    
#     # Create bins for numerical features (deciles)
#     for feature in numerical_features:
#         valid_data = base_df[feature].dropna()
        
#         if len(valid_data) == 0:
#             binning_info[feature] = {'type': 'numerical', 'bins': None, 'bin_ranges': {}}
#             continue
        
#         # Create decile bins
#         try:
#             bins = np.percentile(valid_data, np.arange(0, 101, 10))
#             # Remove duplicates and sort
#             bins = np.unique(bins)
            
#             # IMPROVEMENT: Add infinity edges to capture all values
#             bins = bins.copy()  # Ensure we can modify
#             bins[0] = -np.inf
#             bins[-1] = np.inf
            
#             # Warn if fewer bins than expected due to low variance
#             if len(bins) < 11:  # 11 edges = 10 bins
#                 print(f"Warning: Feature '{feature}' has only {len(bins)-1} bins due to low variance in base month")
            
#             # Create bin ranges dictionary
#             bin_ranges = {}
#             for i in range(len(bins)-1):
#                 bin_name = f"Bin_{i+1}"
#                 bin_ranges[bin_name] = {
#                     'min': bins[i],
#                     'max': bins[i+1],
#                     'range_str': f"[{bins[i]:.2f}, {bins[i+1]:.2f}]" if not np.isinf(bins[i]) and not np.isinf(bins[i+1]) else f"({bins[i]}, {bins[i+1]})"
#                 }
            
#             binning_info[feature] = {
#                 'type': 'numerical',
#                 'bins': bins,
#                 'bin_ranges': bin_ranges
#             }
#         except Exception as e:
#             print(f"Error creating bins for feature '{feature}': {str(e)}")
#             binning_info[feature] = {'type': 'numerical', 'bins': None, 'bin_ranges': {}}
    
#     # Create bins for categorical features (top 6 + others)
#     for feature in categorical_features:
#         value_counts = base_df[feature].value_counts()
#         top_6 = value_counts.nlargest(6).index.tolist()
        
#         binning_info[feature] = {
#             'type': 'categorical',
#             'top_categories': top_6,
#             'bin_ranges': {}  # No ranges for categorical
#         }
    
#     return binning_info


# def apply_binning(df: pd.DataFrame, 
#                   feature: str, 
#                   binning_info: Dict) -> pd.Series:
#     """
#     Apply binning to a feature based on binning information.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     feature : str
#         Feature name
#     binning_info : Dict
#         Binning information for the feature
    
#     Returns:
#     --------
#     pd.Series with binned values
#     """
#     if binning_info['type'] == 'numerical':
#         if binning_info['bins'] is None:
#             return pd.Series(['Missing'] * len(df), index=df.index)
        
#         bins = binning_info['bins']
#         labels = [f"Bin_{i+1}" for i in range(len(bins)-1)]
        
#         binned = pd.cut(df[feature], 
#                        bins=bins, 
#                        labels=labels, 
#                        include_lowest=True,
#                        duplicates='drop')
        
#         # Handle nulls - convert to string and then replace
#         binned = binned.astype(str)
#         binned[df[feature].isna()] = 'Missing'
        
#         return binned
    
#     else:  # categorical
#         top_cats = binning_info['top_categories']
        
#         # Convert categorical to object if needed to avoid category errors
#         if pd.api.types.is_categorical_dtype(df[feature]):
#             feature_data = df[feature].astype(str)
#         else:
#             feature_data = df[feature].astype(str)
        
#         # Replace NaN string representation with 'Missing'
#         feature_data = feature_data.replace('nan', 'Missing')
        
#         # Apply binning logic
#         binned = feature_data.apply(lambda x: x if x in top_cats else ('Missing' if x == 'Missing' else 'Others'))
        
#         return binned


# def calculate_psi(expected_pct: pd.Series, 
#                   actual_pct: pd.Series, 
#                   epsilon: float = 0.0001) -> float:
#     """
#     Calculate Population Stability Index with proper epsilon handling and renormalization.
    
#     Parameters:
#     -----------
#     expected_pct : pd.Series
#         Expected (baseline) percentages
#     actual_pct : pd.Series
#         Actual percentages
#     epsilon : float
#         Small value to avoid log(0)
    
#     Returns:
#     --------
#     PSI value
#     """
#     # Align indices
#     all_bins = expected_pct.index.union(actual_pct.index)
#     expected_pct = expected_pct.reindex(all_bins, fill_value=0)
#     actual_pct = actual_pct.reindex(all_bins, fill_value=0)
    
#     # IMPROVEMENT: Only add epsilon where values are zero
#     expected_pct = expected_pct.apply(lambda x: epsilon if x == 0 else x)
#     actual_pct = actual_pct.apply(lambda x: epsilon if x == 0 else x)
    
#     # IMPROVEMENT: Renormalize to ensure they sum to 1 after adding epsilon
#     expected_pct = expected_pct / expected_pct.sum()
#     actual_pct = actual_pct / actual_pct.sum()
    
#     # Calculate PSI
#     psi_value = np.sum((actual_pct - expected_pct) * np.log(actual_pct / expected_pct))
    
#     return psi_value


# def calculate_month_on_month_psi(df: pd.DataFrame,
#                                  feature_list: List[str],
#                                  segment_columns: List[str],
#                                  month_col: str = 'Application_month') -> pd.DataFrame:
#     """
#     Calculate month-on-month PSI for each feature, overall and by segments.
#     Each row shows PSI comparing first month vs each subsequent month.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     feature_list : List[str]
#         List of features to calculate PSI for
#     segment_columns : List[str]
#         List of segment columns
#     month_col : str
#         Name of month column
    
#     Returns:
#     --------
#     pd.DataFrame with PSI values with one row per feature-month-segment combination
#     """
#     # Create a copy to avoid modifying original
#     df = df.copy()
    
#     # Identify feature types
#     feature_types = identify_feature_types(df, feature_list)
    
#     # Get sorted months
#     months = sorted(df[month_col].unique())
#     base_month = months[0]
    
#     # Create binning strategy based on base month
#     binning_info = create_bins_for_features(
#         df, 
#         feature_types['numerical'],
#         feature_types['categorical'],
#         base_month,
#         month_col
#     )
    
#     results = []
    
#     # Calculate overall PSI
#     for feature in feature_list:
#         if feature not in df.columns:
#             continue
        
#         # Apply binning
#         df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        
#         # Get base month distribution
#         base_dist = df[df[month_col] == base_month][f'{feature}_binned'].value_counts(normalize=True)
        
#         # For each month (including base month for reference)
#         for month in months:
#             actual_dist = df[df[month_col] == month][f'{feature}_binned'].value_counts(normalize=True)
#             psi_value = calculate_psi(base_dist, actual_dist) if month != base_month else 0.0
            
#             results.append({
#                 'Feature': feature,
#                 'Feature_Type': binning_info[feature]['type'],
#                 'Segment_Column': 'Overall',
#                 'Segment_Value': 'All',
#                 'Month': f"{month}",
#                 'Base_Month': base_month,
#                 'Current_Month': month,
#                 'PSI': psi_value
#             })
    
#     # Calculate PSI by segments
#     for segment_col in segment_columns:
#         if segment_col not in df.columns:
#             continue
        
#         segments = df[segment_col].dropna().unique()
        
#         for segment_val in segments:
#             segment_df = df[df[segment_col] == segment_val]
            
#             for feature in feature_list:
#                 if feature not in df.columns:
#                     continue
                
#                 # Get base month distribution for segment
#                 base_segment = segment_df[segment_df[month_col] == base_month]
#                 if len(base_segment) == 0:
#                     continue
                
#                 base_dist = base_segment[f'{feature}_binned'].value_counts(normalize=True)
                
#                 # For each month (including base month for reference)
#                 for month in months:
#                     actual_segment = segment_df[segment_df[month_col] == month]
#                     if len(actual_segment) == 0:
#                         continue
                    
#                     actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)
#                     psi_value = calculate_psi(base_dist, actual_dist) if month != base_month else 0.0
                    
#                     results.append({
#                         'Feature': feature,
#                         'Feature_Type': binning_info[feature]['type'],
#                         'Segment_Column': segment_col,
#                         'Segment_Value': segment_val,
#                         'Month': f"{month}",
#                         'Base_Month': base_month,
#                         'Current_Month': month,
#                         'PSI': psi_value
#                     })
    
#     return pd.DataFrame(results)


# def calculate_bin_level_psi(df: pd.DataFrame,
#                             feature_list: List[str],
#                             segment_columns: List[str],
#                             month_col: str = 'Application_month') -> pd.DataFrame:
#     """
#     Calculate bin-level PSI for each feature, overall and by segments.
#     Each row shows bin-level PSI comparing first month vs each month.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     feature_list : List[str]
#         List of features to calculate PSI for
#     segment_columns : List[str]
#         List of segment columns
#     month_col : str
#         Name of month column
    
#     Returns:
#     --------
#     pd.DataFrame with bin-level PSI details including bin ranges
#     """
#     # Create a copy to avoid modifying original
#     df = df.copy()
    
#     # Identify feature types
#     feature_types = identify_feature_types(df, feature_list)
    
#     # Get sorted months
#     months = sorted(df[month_col].unique())
#     base_month = months[0]
    
#     # Create binning strategy based on base month
#     binning_info = create_bins_for_features(
#         df, 
#         feature_types['numerical'],
#         feature_types['categorical'],
#         base_month,
#         month_col
#     )
    
#     results = []
#     epsilon = 0.0001
    
#     # Calculate overall bin-level PSI
#     for feature in feature_list:
#         if feature not in df.columns:
#             continue
        
#         # Apply binning
#         df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        
#         # Get base month distribution
#         base_dist = df[df[month_col] == base_month][f'{feature}_binned'].value_counts(normalize=True)
        
#         # Calculate bin-level PSI for each month (including base month)
#         for month in months:
#             actual_dist = df[df[month_col] == month][f'{feature}_binned'].value_counts(normalize=True)
            
#             # Get all bins
#             all_bins = base_dist.index.union(actual_dist.index)
            
#             for bin_name in all_bins:
#                 # IMPROVEMENT: Simplified epsilon logic - no redundancy
#                 expected_pct = base_dist.get(bin_name, 0)
#                 actual_pct = actual_dist.get(bin_name, 0)
                
#                 # Add epsilon only if zero
#                 expected_pct = epsilon if expected_pct == 0 else expected_pct
#                 actual_pct = epsilon if actual_pct == 0 else actual_pct
                
#                 # Calculate bin-level PSI
#                 bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct) if month != base_month else 0.0
                
#                 # Get bin range information
#                 bin_ranges = binning_info[feature]['bin_ranges']
#                 if bin_name in bin_ranges:
#                     bin_min = bin_ranges[bin_name]['min']
#                     bin_max = bin_ranges[bin_name]['max']
#                     bin_range = bin_ranges[bin_name]['range_str']
#                 else:
#                     # For categorical or special bins (Missing, Others)
#                     bin_min = None
#                     bin_max = None
#                     bin_range = bin_name
                
#                 results.append({
#                     'Feature': feature,
#                     'Feature_Type': binning_info[feature]['type'],
#                     'Segment_Column': 'Overall',
#                     'Segment_Value': 'All',
#                     'Month': f"{month}",
#                     'Base_Month': base_month,
#                     'Current_Month': month,
#                     'Bin': bin_name,
#                     'Bin_Range': bin_range,
#                     'Bin_Min': bin_min,
#                     'Bin_Max': bin_max,
#                     'Base_Percentage': (base_dist.get(bin_name, 0) * 100),
#                     'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
#                     'Bin_PSI': bin_psi
#                 })
    
#     # Calculate bin-level PSI by segments
#     for segment_col in segment_columns:
#         if segment_col not in df.columns:
#             continue
        
#         segments = df[segment_col].dropna().unique()
        
#         for segment_val in segments:
#             segment_df = df[df[segment_col] == segment_val]
            
#             for feature in feature_list:
#                 if feature not in df.columns:
#                     continue
                
#                 # Get base month distribution for segment
#                 base_segment = segment_df[segment_df[month_col] == base_month]
#                 if len(base_segment) == 0:
#                     continue
                
#                 base_dist = base_segment[f'{feature}_binned'].value_counts(normalize=True)
                
#                 # Calculate bin-level PSI for each month (including base month)
#                 for month in months:
#                     actual_segment = segment_df[segment_df[month_col] == month]
#                     if len(actual_segment) == 0:
#                         continue
                    
#                     actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)
                    
#                     # Get all bins
#                     all_bins = base_dist.index.union(actual_dist.index)
                    
#                     for bin_name in all_bins:
#                         # IMPROVEMENT: Simplified epsilon logic - no redundancy
#                         expected_pct = base_dist.get(bin_name, 0)
#                         actual_pct = actual_dist.get(bin_name, 0)
                        
#                         # Add epsilon only if zero
#                         expected_pct = epsilon if expected_pct == 0 else expected_pct
#                         actual_pct = epsilon if actual_pct == 0 else actual_pct
                        
#                         # Calculate bin-level PSI
#                         bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct) if month != base_month else 0.0
                        
#                         # Get bin range information
#                         bin_ranges = binning_info[feature]['bin_ranges']
#                         if bin_name in bin_ranges:
#                             bin_min = bin_ranges[bin_name]['min']
#                             bin_max = bin_ranges[bin_name]['max']
#                             bin_range = bin_ranges[bin_name]['range_str']
#                         else:
#                             # For categorical or special bins (Missing, Others)
#                             bin_min = None
#                             bin_max = None
#                             bin_range = bin_name
                        
#                         results.append({
#                             'Feature': feature,
#                             'Feature_Type': binning_info[feature]['type'],
#                             'Segment_Column': segment_col,
#                             'Segment_Value': segment_val,
#                             'Month': f"{month}",
#                             'Base_Month': base_month,
#                             'Current_Month': month,
#                             'Bin': bin_name,
#                             'Bin_Range': bin_range,
#                             'Bin_Min': bin_min,
#                             'Bin_Max': bin_max,
#                             'Base_Percentage': (base_dist.get(bin_name, 0) * 100),
#                             'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
#                             'Bin_PSI': bin_psi
#                         })
    
#     return pd.DataFrame(results)

In [6]:
# ## Updated on 27-10-2025 - Modified for Training Period Baseline
# import pandas as pd
# import numpy as np
# from typing import List, Dict, Tuple
# import warnings
# warnings.filterwarnings('ignore')

# def identify_feature_types(df: pd.DataFrame, feature_list: List[str]) -> Dict[str, List[str]]:
#     """
#     Identify categorical and numerical features from the feature list.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     feature_list : List[str]
#         List of features to classify
    
#     Returns:
#     --------
#     Dict with 'categorical' and 'numerical' keys containing respective feature lists
#     """
#     categorical_features = []
#     numerical_features = []
    
#     for feature in feature_list:
#         if feature not in df.columns:
#             print(f"Warning: Feature '{feature}' not found in dataframe")
#             continue
            
#         # Check if feature is numeric
#         if pd.api.types.is_numeric_dtype(df[feature]):
#             # If unique values are less than 15 and all integers, treat as categorical
#             unique_vals = df[feature].nunique()
#             if unique_vals < 15 and df[feature].dropna().apply(lambda x: x == int(x) if isinstance(x, (int, float)) else False).all():
#                 categorical_features.append(feature)
#             else:
#                 numerical_features.append(feature)
#         else:
#             categorical_features.append(feature)
    
#     return {
#         'categorical': categorical_features,
#         'numerical': numerical_features
#     }


# def create_bins_for_features(df: pd.DataFrame, 
#                              numerical_features: List[str],
#                              categorical_features: List[str],
#                              train_period_df: pd.DataFrame) -> Dict:
#     """
#     Create bins for numerical features (deciles) and categorical features (top 6 + others)
#     based on the entire training period data.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Full input dataframe
#     numerical_features : List[str]
#         List of numerical features
#     categorical_features : List[str]
#         List of categorical features
#     train_period_df : pd.DataFrame
#         Training period dataframe (June 2024 to March 2025)
    
#     Returns:
#     --------
#     Dictionary containing binning information for each feature
#     """
#     binning_info = {}
    
#     # Create bins for numerical features (deciles) using training period
#     for feature in numerical_features:
#         valid_data = train_period_df[feature].dropna()
        
#         if len(valid_data) == 0:
#             binning_info[feature] = {'type': 'numerical', 'bins': None, 'bin_ranges': {}}
#             continue
        
#         # Create decile bins
#         try:
#             bins = np.percentile(valid_data, np.arange(0, 101, 10))
#             # Remove duplicates and sort
#             bins = np.unique(bins)
            
#             # Add infinity edges to capture all values
#             bins = bins.copy()
#             bins[0] = -np.inf
#             bins[-1] = np.inf
            
#             # Warn if fewer bins than expected due to low variance
#             if len(bins) < 11:  # 11 edges = 10 bins
#                 print(f"Warning: Feature '{feature}' has only {len(bins)-1} bins due to low variance in training period")
            
#             # Create bin ranges dictionary
#             bin_ranges = {}
#             for i in range(len(bins)-1):
#                 bin_name = f"Bin_{i+1}"
#                 bin_ranges[bin_name] = {
#                     'min': bins[i],
#                     'max': bins[i+1],
#                     'range_str': f"[{bins[i]:.2f}, {bins[i+1]:.2f}]" if not np.isinf(bins[i]) and not np.isinf(bins[i+1]) else f"({bins[i]}, {bins[i+1]})"
#                 }
            
#             binning_info[feature] = {
#                 'type': 'numerical',
#                 'bins': bins,
#                 'bin_ranges': bin_ranges
#             }
#         except Exception as e:
#             print(f"Error creating bins for feature '{feature}': {str(e)}")
#             binning_info[feature] = {'type': 'numerical', 'bins': None, 'bin_ranges': {}}
    
#     # Create bins for categorical features (top 6 + others) using training period
#     for feature in categorical_features:
#         value_counts = train_period_df[feature].value_counts()
#         top_6 = value_counts.nlargest(6).index.tolist()
        
#         binning_info[feature] = {
#             'type': 'categorical',
#             'top_categories': top_6,
#             'bin_ranges': {}  # No ranges for categorical
#         }
    
#     return binning_info


# def apply_binning(df: pd.DataFrame, 
#                   feature: str, 
#                   binning_info: Dict) -> pd.Series:
#     """
#     Apply binning to a feature based on binning information.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     feature : str
#         Feature name
#     binning_info : Dict
#         Binning information for the feature
    
#     Returns:
#     --------
#     pd.Series with binned values
#     """
#     if binning_info['type'] == 'numerical':
#         if binning_info['bins'] is None:
#             return pd.Series(['Missing'] * len(df), index=df.index)
        
#         bins = binning_info['bins']
#         labels = [f"Bin_{i+1}" for i in range(len(bins)-1)]
        
#         binned = pd.cut(df[feature], 
#                        bins=bins, 
#                        labels=labels, 
#                        include_lowest=True,
#                        duplicates='drop')
        
#         # Handle nulls - convert to string and then replace
#         binned = binned.astype(str)
#         binned[df[feature].isna()] = 'Missing'
        
#         return binned
    
#     else:  # categorical
#         top_cats = binning_info['top_categories']
        
#         # Convert categorical to object if needed to avoid category errors
#         if pd.api.types.is_categorical_dtype(df[feature]):
#             feature_data = df[feature].astype(str)
#         else:
#             feature_data = df[feature].astype(str)
        
#         # Replace NaN string representation with 'Missing'
#         feature_data = feature_data.replace('nan', 'Missing')
        
#         # Apply binning logic
#         binned = feature_data.apply(lambda x: x if x in top_cats else ('Missing' if x == 'Missing' else 'Others'))
        
#         return binned


# def calculate_psi(expected_pct: pd.Series, 
#                   actual_pct: pd.Series, 
#                   epsilon: float = 0.0001) -> float:
#     """
#     Calculate Population Stability Index with proper epsilon handling and renormalization.
    
#     Parameters:
#     -----------
#     expected_pct : pd.Series
#         Expected (baseline) percentages
#     actual_pct : pd.Series
#         Actual percentages
#     epsilon : float
#         Small value to avoid log(0)
    
#     Returns:
#     --------
#     PSI value
#     """
#     # Align indices
#     all_bins = expected_pct.index.union(actual_pct.index)
#     expected_pct = expected_pct.reindex(all_bins, fill_value=0)
#     actual_pct = actual_pct.reindex(all_bins, fill_value=0)
    
#     # Only add epsilon where values are zero
#     expected_pct = expected_pct.apply(lambda x: epsilon if x == 0 else x)
#     actual_pct = actual_pct.apply(lambda x: epsilon if x == 0 else x)
    
#     # Renormalize to ensure they sum to 1 after adding epsilon
#     expected_pct = expected_pct / expected_pct.sum()
#     actual_pct = actual_pct / actual_pct.sum()
    
#     # Calculate PSI
#     psi_value = np.sum((actual_pct - expected_pct) * np.log(actual_pct / expected_pct))
    
#     return psi_value


# def calculate_month_on_month_psi(df: pd.DataFrame,
#                                  feature_list: List[str],
#                                  segment_columns: List[str],
#                                  month_col: str = 'Application_month',
#                                  data_selection_col: str = 'Data_selection') -> pd.DataFrame:
#     """
#     Calculate PSI for each feature comparing training period (June 2024 to March 2025) 
#     vs each month after March 2025, overall and by segments.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     feature_list : List[str]
#         List of features to calculate PSI for
#     segment_columns : List[str]
#         List of segment columns
#     month_col : str
#         Name of month column
#     data_selection_col : str
#         Name of data selection column (identifies train period)
    
#     Returns:
#     --------
#     pd.DataFrame with PSI values with one row per feature-month-segment combination
#     """
#     # Create a copy to avoid modifying original
#     df = df.copy()
    
#     # Identify training and test periods
#     train_df = df[df[data_selection_col] == 'Train'].copy()
#     test_df = df[df[data_selection_col] != 'Train'].copy()
    
#     if len(train_df) == 0:
#         raise ValueError("No training data found. Check Data_selection column.")
    
#     print(f"Training period: {train_df[month_col].min()} to {train_df[month_col].max()}")
#     print(f"Test period: {test_df[month_col].min()} to {test_df[month_col].max()}")
    
#     # Identify feature types
#     feature_types = identify_feature_types(df, feature_list)
    
#     # Create binning strategy based on training period
#     binning_info = create_bins_for_features(
#         df, 
#         feature_types['numerical'],
#         feature_types['categorical'],
#         train_df
#     )
    
#     # Get sorted test months
#     test_months = sorted(test_df[month_col].unique())
    
#     results = []
    
#     # Calculate overall PSI
#     for feature in feature_list:
#         if feature not in df.columns:
#             continue
        
#         # Apply binning to entire dataset
#         df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        
#         # Get training period distribution (baseline)
#         train_baseline = df[df[data_selection_col] == 'Train'][f'{feature}_binned'].value_counts(normalize=True)
        
#         # Calculate PSI for each test month
#         for month in test_months:
#             actual_dist = df[df[month_col] == month][f'{feature}_binned'].value_counts(normalize=True)
#             psi_value = calculate_psi(train_baseline, actual_dist)
            
#             results.append({
#                 'Feature': feature,
#                 'Feature_Type': binning_info[feature]['type'],
#                 'Segment_Column': 'Overall',
#                 'Segment_Value': 'All',
#                 'Month': f"{month}",
#                 'Base_Period': 'Train (Jun 2024 - Mar 2025)',
#                 'Current_Month': month,
#                 'PSI': psi_value
#             })
    
#     # Calculate PSI by segments
#     for segment_col in segment_columns:
#         if segment_col not in df.columns:
#             continue
        
#         segments = df[segment_col].dropna().unique()
        
#         for segment_val in segments:
#             segment_df = df[df[segment_col] == segment_val]
            
#             for feature in feature_list:
#                 if feature not in df.columns:
#                     continue
                
#                 # Get training period distribution for segment
#                 train_segment = segment_df[segment_df[data_selection_col] == 'Train']
#                 if len(train_segment) == 0:
#                     continue
                
#                 train_baseline = train_segment[f'{feature}_binned'].value_counts(normalize=True)
                
#                 # Calculate PSI for each test month
#                 for month in test_months:
#                     actual_segment = segment_df[segment_df[month_col] == month]
#                     if len(actual_segment) == 0:
#                         continue
                    
#                     actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)
#                     psi_value = calculate_psi(train_baseline, actual_dist)
                    
#                     results.append({
#                         'Feature': feature,
#                         'Feature_Type': binning_info[feature]['type'],
#                         'Segment_Column': segment_col,
#                         'Segment_Value': segment_val,
#                         'Month': f"{month}",
#                         'Base_Month': 'Train (Jun 2024 - Mar 2025)',
#                         'Current_Month': month,
#                         'PSI': psi_value
#                     })
    
#     return pd.DataFrame(results)


# def calculate_bin_level_psi(df: pd.DataFrame,
#                             feature_list: List[str],
#                             segment_columns: List[str],
#                             month_col: str = 'Application_month',
#                             data_selection_col: str = 'Data_selection') -> pd.DataFrame:
#     """
#     Calculate bin-level PSI for each feature comparing training period 
#     vs each month after March 2025, overall and by segments.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     feature_list : List[str]
#         List of features to calculate PSI for
#     segment_columns : List[str]
#         List of segment columns
#     month_col : str
#         Name of month column
#     data_selection_col : str
#         Name of data selection column
    
#     Returns:
#     --------
#     pd.DataFrame with bin-level PSI details including bin ranges
#     """
#     # Create a copy to avoid modifying original
#     df = df.copy()
    
#     # Identify training and test periods
#     train_df = df[df[data_selection_col] == 'Train'].copy()
#     test_df = df[df[data_selection_col] != 'Train'].copy()
    
#     if len(train_df) == 0:
#         raise ValueError("No training data found. Check Data_selection column.")
    
#     print(f"Training period: {train_df[month_col].min()} to {train_df[month_col].max()}")
#     print(f"Test period: {test_df[month_col].min()} to {test_df[month_col].max()}")
    
#     # Identify feature types
#     feature_types = identify_feature_types(df, feature_list)
    
#     # Create binning strategy based on training period
#     binning_info = create_bins_for_features(
#         df, 
#         feature_types['numerical'],
#         feature_types['categorical'],
#         train_df
#     )
    
#     # Get sorted test months
#     test_months = sorted(test_df[month_col].unique())
    
#     results = []
#     epsilon = 0.0001
    
#     # Calculate overall bin-level PSI
#     for feature in feature_list:
#         if feature not in df.columns:
#             continue
        
#         # Apply binning to entire dataset
#         df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        
#         # Get training period distribution (baseline)
#         train_baseline = df[df[data_selection_col] == 'Train'][f'{feature}_binned'].value_counts(normalize=True)
        
#         # Calculate bin-level PSI for each test month
#         for month in test_months:
#             actual_dist = df[df[month_col] == month][f'{feature}_binned'].value_counts(normalize=True)
            
#             # Get all bins
#             all_bins = train_baseline.index.union(actual_dist.index)
            
#             for bin_name in all_bins:
#                 # Simplified epsilon logic - no redundancy
#                 expected_pct = train_baseline.get(bin_name, 0)
#                 actual_pct = actual_dist.get(bin_name, 0)
                
#                 # Add epsilon only if zero
#                 expected_pct = epsilon if expected_pct == 0 else expected_pct
#                 actual_pct = epsilon if actual_pct == 0 else actual_pct
                
#                 # Calculate bin-level PSI
#                 bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct)
                
#                 # Get bin range information
#                 bin_ranges = binning_info[feature]['bin_ranges']
#                 if bin_name in bin_ranges:
#                     bin_min = bin_ranges[bin_name]['min']
#                     bin_max = bin_ranges[bin_name]['max']
#                     bin_range = bin_ranges[bin_name]['range_str']
#                 else:
#                     # For categorical or special bins (Missing, Others)
#                     bin_min = None
#                     bin_max = None
#                     bin_range = bin_name
                
#                 results.append({
#                     'Feature': feature,
#                     'Feature_Type': binning_info[feature]['type'],
#                     'Segment_Column': 'Overall',
#                     'Segment_Value': 'All',
#                     'Month': f"{month}",
#                     'Base_Month': 'Train (Jun 2024 - Mar 2025)',
#                     'Current_Month': month,
#                     'Bin': bin_name,
#                     'Bin_Range': bin_range,
#                     'Bin_Min': bin_min,
#                     'Bin_Max': bin_max,
#                     'Base_Percentage': (train_baseline.get(bin_name, 0) * 100),
#                     'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
#                     'Bin_PSI': bin_psi
#                 })
    
#     # Calculate bin-level PSI by segments
#     for segment_col in segment_columns:
#         if segment_col not in df.columns:
#             continue
        
#         segments = df[segment_col].dropna().unique()
        
#         for segment_val in segments:
#             segment_df = df[df[segment_col] == segment_val]
            
#             for feature in feature_list:
#                 if feature not in df.columns:
#                     continue
                
#                 # Get training period distribution for segment
#                 train_segment = segment_df[segment_df[data_selection_col] == 'Train']
#                 if len(train_segment) == 0:
#                     continue
                
#                 train_baseline = train_segment[f'{feature}_binned'].value_counts(normalize=True)
                
#                 # Calculate bin-level PSI for each test month
#                 for month in test_months:
#                     actual_segment = segment_df[segment_df[month_col] == month]
#                     if len(actual_segment) == 0:
#                         continue
                    
#                     actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)
                    
#                     # Get all bins
#                     all_bins = train_baseline.index.union(actual_dist.index)
                    
#                     for bin_name in all_bins:
#                         # Simplified epsilon logic - no redundancy
#                         expected_pct = train_baseline.get(bin_name, 0)
#                         actual_pct = actual_dist.get(bin_name, 0)
                        
#                         # Add epsilon only if zero
#                         expected_pct = epsilon if expected_pct == 0 else expected_pct
#                         actual_pct = epsilon if actual_pct == 0 else actual_pct
                        
#                         # Calculate bin-level PSI
#                         bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct)
                        
#                         # Get bin range information
#                         bin_ranges = binning_info[feature]['bin_ranges']
#                         if bin_name in bin_ranges:
#                             bin_min = bin_ranges[bin_name]['min']
#                             bin_max = bin_ranges[bin_name]['max']
#                             bin_range = bin_ranges[bin_name]['range_str']
#                         else:
#                             # For categorical or special bins (Missing, Others)
#                             bin_min = None
#                             bin_max = None
#                             bin_range = bin_name
                        
#                         results.append({
#                             'Feature': feature,
#                             'Feature_Type': binning_info[feature]['type'],
#                             'Segment_Column': segment_col,
#                             'Segment_Value': segment_val,
#                             'Month': f"{month}",
#                             'Base_Month': 'Train (Jun 2024 - Mar 2025)',
#                             'Current_Month': month,
#                             'Bin': bin_name,
#                             'Bin_Range': bin_range,
#                             'Bin_Min': bin_min,
#                             'Bin_Max': bin_max,
#                             'Base_Percentage': (train_baseline.get(bin_name, 0) * 100),
#                             'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
#                             'Bin_PSI': bin_psi
#                         })
    
#     return pd.DataFrame(results)

# Queries

## SIL

### Alpha - CIC-SIL-Model

#### Query from ml_model_run_details

In [7]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,

    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName = 'Alpha - CIC-SIL-Model')
SELECT
  r.customerId,r.digitalLoanAccountId,prediction Alpha_cic_sil_score
    ,start_time,end_time,modelDisplayName,modelVersionId,
   loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
        when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
        when lower(loanmaster.deviceType) like '%andro%' then 'android'
        else 'ios' end osType,
 'Alpha - CIC-SIL-Model' Model_Name,
 'SIL' as product,
 'NA' trenchCategory,
  r.calcFeature calcFeatures,
  'Test' Data_selection,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,  
FROM cleaned r 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
;
"""

dfd = client.query(sq).to_dataframe()
print(f"The shape of the dataframe is: {dfd.shape}")
dfd.head()

The shape of the dataframe is: (68874, 19)


,customerId,digitalLoanAccountId,Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,Data_selection,appln_submit_datetime,disbursementDateTime,Application_month
0,1544765,2ae5693f-4e55-4ecd-85ff-8663c18b77a6,0.05340595632563326,2025-07-31 08:26:35.251431,2025-07-31 08:26:35.257667,Alpha - CIC-SIL-Model,v1,SIL-Instore,F,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1753920000000,""cic_Personal_Loans_...",Test,2025-07-31 16:26:25,2025-07-31 16:31:31,2025-07
1,2195641,b762fc41-4484-4967-be42-d9a9b9a31abe,0.09586900094807521,2025-07-10 04:52:54.229887,2025-07-10 04:52:54.236271,Alpha - CIC-SIL-Model,v1,SIL-Instore,F,Appliance,ios,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1752105600000,""cic_Personal_Loans_...",Test,2025-07-10 12:52:43,NaT,2025-07
2,2197627,537b2534-7346-4c7d-8427-e493e560226b,0.11051768448845367,2025-06-25 08:20:43.350939,2025-06-25 08:20:43.357278,Alpha - CIC-SIL-Model,v1,SIL Competitor,F,Mall,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1750809600000,""cic_Personal_Loans_...",Test,2025-06-25 16:20:33,NaT,2025-06
3,2262515,b3b21dac-a006-4ebf-8b3c-30e32da2b44b,0.14626422034322584,2025-07-17 03:28:02.093331,2025-07-17 03:28:02.099551,Alpha - CIC-SIL-Model,v1,SIL-Instore,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1752710400000,""cic_Personal_Loans_...",Test,2025-07-17 11:27:51,NaT,2025-07
4,2318373,43ab6c5f-6d2f-4984-8bbe-a3e16b4d722e,0.05568168914472758,2025-08-09 10:08:41.831246,2025-08-09 10:08:41.837609,Alpha - CIC-SIL-Model,v1,SIL Competitor,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1754697600000,""cic_Personal_Loans_...",Test,2025-08-09 18:08:29,2025-08-09 18:12:20,2025-08


##### expanded_df

In [8]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 19
Expanded columns: 29


,customerId,digitalLoanAccountId,Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,Data_selection,appln_submit_datetime,disbursementDateTime,Application_month,calc_run_date,calc_cic_Personal_Loans_granted_contracts_amt_24M,calc_cic_days_since_last_inquiry,calc_cic_cnt_active_contracts,calc_cic_vel_contract_nongranted_cnt_12on24,calc_cic_max_amt_granted_24M,calc_cic_zero_non_granted_ever_flag,calc_cic_tot_active_contracts_util,calc_cic_vel_contract_granted_amt_12on24,calc_cic_zero_granted_ever_flag
0,1544765,2ae5693f-4e55-4ecd-85ff-8663c18b77a6,0.05340595632563326,2025-07-31 08:26:35.251431,2025-07-31 08:26:35.257667,Alpha - CIC-SIL-Model,v1,SIL-Instore,F,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1753920000000,""cic_Personal_Loans_...",Test,2025-07-31 16:26:25,2025-07-31 16:31:31,2025-07,1.753920e+12,67400.0,134.0,2.0,NaN,88000.0,1,0.556750,1.671148,0
1,2195641,b762fc41-4484-4967-be42-d9a9b9a31abe,0.09586900094807521,2025-07-10 04:52:54.229887,2025-07-10 04:52:54.236271,Alpha - CIC-SIL-Model,v1,SIL-Instore,F,Appliance,ios,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1752105600000,""cic_Personal_Loans_...",Test,2025-07-10 12:52:43,NaT,2025-07,1.752106e+12,6839.0,239.0,NaN,2.000000,7100.0,0,NaN,1.249092,0
2,2197627,537b2534-7346-4c7d-8427-e493e560226b,0.11051768448845367,2025-06-25 08:20:43.350939,2025-06-25 08:20:43.357278,Alpha - CIC-SIL-Model,v1,SIL Competitor,F,Mall,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1750809600000,""cic_Personal_Loans_...",Test,2025-06-25 16:20:33,NaT,2025-06,1.750810e+12,2010.0,85.0,5.0,2.012048,7000.0,0,0.001965,1.000000,0
3,2262515,b3b21dac-a006-4ebf-8b3c-30e32da2b44b,0.14626422034322584,2025-07-17 03:28:02.093331,2025-07-17 03:28:02.099551,Alpha - CIC-SIL-Model,v1,SIL-Instore,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1752710400000,""cic_Personal_Loans_...",Test,2025-07-17 11:27:51,NaT,2025-07,1.752710e+12,2800.0,321.0,1.0,1.201923,2800.0,0,0.000000,NaN,0
4,2318373,43ab6c5f-6d2f-4984-8bbe-a3e16b4d722e,0.05568168914472758,2025-08-09 10:08:41.831246,2025-08-09 10:08:41.837609,Alpha - CIC-SIL-Model,v1,SIL Competitor,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1754697600000,""cic_Personal_Loans_...",Test,2025-08-09 18:08:29,2025-08-09 18:12:20,2025-08,1.754698e+12,77336.0,241.0,13.0,0.664000,47336.0,0,1.013675,0.260520,0


In [9]:
expanded_df.groupby('Application_month').size()

Application_month
2025-03      180
2025-04      541
2025-05      858
2025-06     9934
2025-07    14043
2025-08    15085
2025-09    13880
2025-10    14353
dtype: int64

## Query from risk_mart.sil_risk_ds_master_20230101_20250309_v2

In [10]:
sq = """ 
with base as 
(select 
r.customerId,
r.digitalLoanAccountId,
r.cic_score prediction,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) start_time,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) end_time,
'Alpha-Cash-CIC-Model' modelDisplayName,
'v1' modelVersionId,
loanmaster.new_loan_type,
loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
  when lower(loanmaster.deviceType) like '%andro%' then 'android'
  else 'ios' end osType,
'SIL' as product,
r.cic_score Alpha_cic_sil_score,  
r.cic_called_flag,
r.cic_hit_flag,
r.cic_Personal_Loans_granted_contracts_amt_24M,
r.cic_days_since_last_inquiry,
r.cic_cnt_active_contracts,
r.cic_vel_contract_nongranted_cnt_12on24,
r.cic_max_amt_granted_24M,
r.cic_zero_non_granted_ever_flag,
r.cic_tot_active_contracts_util,
r.cic_vel_contract_granted_amt_12on24,
r.cic_zero_granted_ever_flag,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
loanmaster.disbursementDateTime,
format_date('%Y-%m', IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month, 
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
where cic_score is not null
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) desc) = 1
)
select 
  base.customerId,
  base.digitalLoanAccountId,
  base.Alpha_cic_sil_score,
  base.start_time,
  base.end_time,
  base.modelDisplayName,
  base.modelVersionId,
  base.new_loan_type,
  base.gender,
  base.loan_product_type,
  base.osType,
  'Alpha - CIC-SIL-Model' Model_Name,
  base.product,
  'NA' trenchCategory,
  ''calcFeatures,
  appln_submit_datetime,
  disbursementDateTime,
  Application_month,
  ' ' calc_run_date,
  'Train' Data_selection,
  base.cic_Personal_Loans_granted_contracts_amt_24M calc_cic_Personal_Loans_granted_contracts_amt_24M,
  base.cic_days_since_last_inquiry calc_cic_days_since_last_inquiry,
  base.cic_cnt_active_contracts calc_cic_cnt_active_contracts,
  base.cic_vel_contract_nongranted_cnt_12on24 calc_cic_vel_contract_nongranted_cnt_12on24,
  base.cic_max_amt_granted_24M calc_cic_max_amt_granted_24M,
  base.cic_zero_non_granted_ever_flag calc_cic_zero_non_granted_ever_flag,
  base.cic_tot_active_contracts_util calc_cic_tot_active_contracts_util,
  base.cic_vel_contract_granted_amt_12on24 calc_cic_vel_contract_granted_amt_12on24,
  base.cic_zero_granted_ever_flag calc_cic_zero_granted_ever_flag
from base
Where date(appln_submit_datetime) >= '2024-06-01'
;
""" 
df1 = client.query(sq).to_dataframe(progress_bar_type="tqdm")
print(f"The shape of the dataframe is: {df1.shape}")
df1.head()



Job ID a886166f-1261-429c-9a8d-c5fa2522ebb7 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is: (167859, 29)


,customerId,digitalLoanAccountId,Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_run_date,Data_selection,calc_cic_Personal_Loans_granted_contracts_amt_24M,calc_cic_days_since_last_inquiry,calc_cic_cnt_active_contracts,calc_cic_vel_contract_nongranted_cnt_12on24,calc_cic_max_amt_granted_24M,calc_cic_zero_non_granted_ever_flag,calc_cic_tot_active_contracts_util,calc_cic_vel_contract_granted_amt_12on24,calc_cic_zero_granted_ever_flag
0,1513030,0cae16e9-255f-4a5f-be12-94e456d653aa,0.059278,2024-12-21 16:46:02,2024-12-21 16:46:02,Alpha-Cash-CIC-Model,v1,SIL Repeat,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,,2024-12-21 16:46:02,2024-12-21 16:49:15,2024-12,,Train,85000.0,144.0,4.0,NaN,64270.0,0,0.941996,0.529684,0
1,1753158,670356a6-6d1e-4d67-8162-8833c435245f,0.123014,2024-09-01 09:35:36,2024-09-01 09:35:36,Alpha-Cash-CIC-Model,v1,SIL-Instore,M,Appliance,ios,Alpha - CIC-SIL-Model,SIL,NA,,2024-09-01 09:35:36,NaT,2024-09,,Train,NaN,1052.0,NaN,NaN,NaN,1,NaN,NaN,0
2,2034347,eaa3d5c3-fe57-43f6-8a5d-7e33e67b2193,0.154300,2024-11-02 11:22:21,2024-11-02 11:22:21,Alpha-Cash-CIC-Model,v1,SIL-Instore,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,,2024-11-02 11:22:21,2024-11-02 11:26:29,2024-11,,Train,NaN,393.0,1.0,NaN,NaN,0,NaN,NaN,0
3,2034650,4712256f-4836-44cc-8df9-082333fef58a,0.158599,2024-06-22 09:36:51,2024-06-22 09:36:51,Alpha-Cash-CIC-Model,v1,SIL-Instore,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,,2024-06-22 09:36:51,NaT,2024-06,,Train,NaN,310.0,NaN,1.97619,NaN,0,NaN,NaN,1
4,2122971,28d200ea-958a-4a5c-8fff-e4631b242cd4,0.052806,2025-02-12 19:13:28,2025-02-12 19:13:28,Alpha-Cash-CIC-Model,v1,SIL-Instore,F,Appliance,ios,Alpha - CIC-SIL-Model,SIL,NA,,2025-02-12 19:13:28,2025-02-12 19:24:42,2025-02,,Train,148999.0,74.0,10.0,1.97619,50000.0,0,0.167904,1.224388,0


In [11]:
df1.groupby('Application_month').size()

Application_month
2024-06    13472
2024-07    13346
2024-08    17808
2024-09    17502
2024-10    16817
2024-11    17931
2024-12    34696
2025-01    16804
2025-02    14246
2025-03     5237
dtype: int64

## concatenation

In [12]:
df_concat = pd.concat([expanded_df, df1], ignore_index=True)
df_concat.head()

,customerId,digitalLoanAccountId,Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,Data_selection,appln_submit_datetime,disbursementDateTime,Application_month,calc_run_date,calc_cic_Personal_Loans_granted_contracts_amt_24M,calc_cic_days_since_last_inquiry,calc_cic_cnt_active_contracts,calc_cic_vel_contract_nongranted_cnt_12on24,calc_cic_max_amt_granted_24M,calc_cic_zero_non_granted_ever_flag,calc_cic_tot_active_contracts_util,calc_cic_vel_contract_granted_amt_12on24,calc_cic_zero_granted_ever_flag
0,1544765,2ae5693f-4e55-4ecd-85ff-8663c18b77a6,0.05340595632563326,2025-07-31 08:26:35.251431,2025-07-31 08:26:35.257667,Alpha - CIC-SIL-Model,v1,SIL-Instore,F,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1753920000000,""cic_Personal_Loans_...",Test,2025-07-31 16:26:25,2025-07-31 16:31:31,2025-07,1753920000000.0,67400.0,134.0,2.0,NaN,88000.0,1,0.556750,1.671148,0
1,2195641,b762fc41-4484-4967-be42-d9a9b9a31abe,0.09586900094807521,2025-07-10 04:52:54.229887,2025-07-10 04:52:54.236271,Alpha - CIC-SIL-Model,v1,SIL-Instore,F,Appliance,ios,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1752105600000,""cic_Personal_Loans_...",Test,2025-07-10 12:52:43,NaT,2025-07,1752105600000.0,6839.0,239.0,NaN,2.000000,7100.0,0,NaN,1.249092,0
2,2197627,537b2534-7346-4c7d-8427-e493e560226b,0.11051768448845367,2025-06-25 08:20:43.350939,2025-06-25 08:20:43.357278,Alpha - CIC-SIL-Model,v1,SIL Competitor,F,Mall,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1750809600000,""cic_Personal_Loans_...",Test,2025-06-25 16:20:33,NaT,2025-06,1750809600000.0,2010.0,85.0,5.0,2.012048,7000.0,0,0.001965,1.000000,0
3,2262515,b3b21dac-a006-4ebf-8b3c-30e32da2b44b,0.14626422034322584,2025-07-17 03:28:02.093331,2025-07-17 03:28:02.099551,Alpha - CIC-SIL-Model,v1,SIL-Instore,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1752710400000,""cic_Personal_Loans_...",Test,2025-07-17 11:27:51,NaT,2025-07,1752710400000.0,2800.0,321.0,1.0,1.201923,2800.0,0,0.000000,NaN,0
4,2318373,43ab6c5f-6d2f-4984-8bbe-a3e16b4d722e,0.05568168914472758,2025-08-09 10:08:41.831246,2025-08-09 10:08:41.837609,Alpha - CIC-SIL-Model,v1,SIL Competitor,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1754697600000,""cic_Personal_Loans_...",Test,2025-08-09 18:08:29,2025-08-09 18:12:20,2025-08,1754697600000.0,77336.0,241.0,13.0,0.664000,47336.0,0,1.013675,0.260520,0


In [13]:
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")

The shape of the concatenated dataframe is: (236733, 29)


In [14]:
df_concat.groupby(['Data_selection','Application_month'])['digitalLoanAccountId'].nunique().reset_index().sort_values(by=['Application_month','Data_selection'])

,Data_selection,Application_month,digitalLoanAccountId
8,Train,2024-06,13472
9,Train,2024-07,13346
10,Train,2024-08,17808
11,Train,2024-09,17502
12,Train,2024-10,16817
13,Train,2024-11,17931
14,Train,2024-12,34696
15,Train,2025-01,16804
16,Train,2025-02,14246
0,Test,2025-03,180


In [15]:
df_concat.to_csv('cic_sil_model_monitoring_data.csv', index=False)

In [16]:
df_concat.columns

Index(['customerId', 'digitalLoanAccountId', 'Alpha_cic_sil_score',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
       'new_loan_type', 'gender', 'loan_product_type', 'osType', 'Model_Name',
       'product', 'trenchCategory', 'calcFeatures', 'Data_selection',
       'appln_submit_datetime', 'disbursementDateTime', 'Application_month',
       'calc_run_date', 'calc_cic_Personal_Loans_granted_contracts_amt_24M',
       'calc_cic_days_since_last_inquiry', 'calc_cic_cnt_active_contracts',
       'calc_cic_vel_contract_nongranted_cnt_12on24',
       'calc_cic_max_amt_granted_24M', 'calc_cic_zero_non_granted_ever_flag',
       'calc_cic_tot_active_contracts_util',
       'calc_cic_vel_contract_granted_amt_12on24',
       'calc_cic_zero_granted_ever_flag'],
      dtype='object')

In [17]:
df = df_concat.drop(columns=['calcFeatures']).copy()

In [18]:
df['Alpha_cic_sil_score'] = pd.to_numeric(df['Alpha_cic_sil_score'], errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236733 entries, 0 to 236732
Data columns (total 28 columns):
 #   Column                                             Non-Null Count   Dtype         
---  ------                                             --------------   -----         
 0   customerId                                         236733 non-null  object        
 1   digitalLoanAccountId                               236733 non-null  object        
 2   Alpha_cic_sil_score                                236733 non-null  float64       
 3   start_time                                         236733 non-null  datetime64[us]
 4   end_time                                           236733 non-null  datetime64[us]
 5   modelDisplayName                                   236733 non-null  object        
 6   modelVersionId                                     236733 non-null  object        
 7   new_loan_type                                      236571 non-null  object        
 8   gend

## PSI calculation

In [19]:
df = df.copy()

# Define feature list
feature_list = ['Alpha_cic_sil_score',  'calc_cic_Personal_Loans_granted_contracts_amt_24M',
       'calc_cic_days_since_last_inquiry', 'calc_cic_cnt_active_contracts',
       'calc_cic_vel_contract_nongranted_cnt_12on24',
       'calc_cic_max_amt_granted_24M', 'calc_cic_zero_non_granted_ever_flag',
       'calc_cic_tot_active_contracts_util',
       'calc_cic_vel_contract_granted_amt_12on24',
       'calc_cic_zero_granted_ever_flag']

# Define segment columns
segment_columns = ['new_loan_type','osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                           'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
                           'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                           'Expected_Percentage', 'Actual_Percentage', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                                   'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value',
                                    'Month', 'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                                    'Bin', 'Bin_Range', 'Base_Percentage', 'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cic_sil_model_psi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cic_sil_model_csi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

Training period: 2024-06 to 2025-03
Test period: 2025-03 to 2025-10
Bin Ranges for feature 'Alpha_cic_sil_score': {'Bin_1': {'min': -inf, 'max': 0.07281993293225933, 'range_str': '(-inf, 0.07281993293225933)'}, 'Bin_2': {'min': 0.07281993293225933, 'max': 0.0972321781725312, 'range_str': '[0.07, 0.10]'}, 'Bin_3': {'min': 0.0972321781725312, 'max': 0.11686161296298704, 'range_str': '[0.10, 0.12]'}, 'Bin_4': {'min': 0.11686161296298704, 'max': 0.1254825741542581, 'range_str': '[0.12, 0.13]'}, 'Bin_5': {'min': 0.1254825741542581, 'max': 0.13216345408198324, 'range_str': '[0.13, 0.13]'}, 'Bin_6': {'min': 0.13216345408198324, 'max': 0.15733181863629542, 'range_str': '[0.13, 0.16]'}, 'Bin_7': {'min': 0.15733181863629542, 'max': 0.16897199708566576, 'range_str': '[0.16, 0.17]'}, 'Bin_8': {'min': 0.16897199708566576, 'max': 0.1933502863359173, 'range_str': '[0.17, 0.19]'}, 'Bin_9': {'min': 0.1933502863359173, 'max': 0.20131316653565443, 'range_str': '[0.19, 0.20]'}, 'Bin_10': {'min': 0.2013131

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=01d4c959-6432-41d5-a84c-55462a012d6b>

In [20]:
df_concat.head()

,customerId,digitalLoanAccountId,Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,Data_selection,appln_submit_datetime,disbursementDateTime,Application_month,calc_run_date,calc_cic_Personal_Loans_granted_contracts_amt_24M,calc_cic_days_since_last_inquiry,calc_cic_cnt_active_contracts,calc_cic_vel_contract_nongranted_cnt_12on24,calc_cic_max_amt_granted_24M,calc_cic_zero_non_granted_ever_flag,calc_cic_tot_active_contracts_util,calc_cic_vel_contract_granted_amt_12on24,calc_cic_zero_granted_ever_flag
0,1544765,2ae5693f-4e55-4ecd-85ff-8663c18b77a6,0.05340595632563326,2025-07-31 08:26:35.251431,2025-07-31 08:26:35.257667,Alpha - CIC-SIL-Model,v1,SIL-Instore,F,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1753920000000,""cic_Personal_Loans_...",Test,2025-07-31 16:26:25,2025-07-31 16:31:31,2025-07,1753920000000.0,67400.0,134.0,2.0,NaN,88000.0,1,0.556750,1.671148,0
1,2195641,b762fc41-4484-4967-be42-d9a9b9a31abe,0.09586900094807521,2025-07-10 04:52:54.229887,2025-07-10 04:52:54.236271,Alpha - CIC-SIL-Model,v1,SIL-Instore,F,Appliance,ios,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1752105600000,""cic_Personal_Loans_...",Test,2025-07-10 12:52:43,NaT,2025-07,1752105600000.0,6839.0,239.0,NaN,2.000000,7100.0,0,NaN,1.249092,0
2,2197627,537b2534-7346-4c7d-8427-e493e560226b,0.11051768448845367,2025-06-25 08:20:43.350939,2025-06-25 08:20:43.357278,Alpha - CIC-SIL-Model,v1,SIL Competitor,F,Mall,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1750809600000,""cic_Personal_Loans_...",Test,2025-06-25 16:20:33,NaT,2025-06,1750809600000.0,2010.0,85.0,5.0,2.012048,7000.0,0,0.001965,1.000000,0
3,2262515,b3b21dac-a006-4ebf-8b3c-30e32da2b44b,0.14626422034322584,2025-07-17 03:28:02.093331,2025-07-17 03:28:02.099551,Alpha - CIC-SIL-Model,v1,SIL-Instore,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1752710400000,""cic_Personal_Loans_...",Test,2025-07-17 11:27:51,NaT,2025-07,1752710400000.0,2800.0,321.0,1.0,1.201923,2800.0,0,0.000000,NaN,0
4,2318373,43ab6c5f-6d2f-4984-8bbe-a3e16b4d722e,0.05568168914472758,2025-08-09 10:08:41.831246,2025-08-09 10:08:41.837609,Alpha - CIC-SIL-Model,v1,SIL Competitor,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1754697600000,""cic_Personal_Loans_...",Test,2025-08-09 18:08:29,2025-08-09 18:12:20,2025-08,1754697600000.0,77336.0,241.0,13.0,0.664000,47336.0,0,1.013675,0.260520,0
